## Test Scrape Notebook

In [3]:
## Import library as establish paths
from pypdf import PdfReader

filename = 'Kronijk_van_het_Historisch_Genootschap_g'

source_pdf_input_path = r'./input/' + filename + r'.pdf'
source_text_output_path = r'./output/' + filename + '.txt'
trans_text_output_path = r'./output/' + filename + '_translated.txt'


In [4]:
reader = PdfReader(source_pdf_input_path)
number_of_pages = len(reader.pages)
page = reader.pages[0]
text = page.extract_text()

In [8]:
print(number_of_pages - 856)
# Pulling pages 775 to 830

16


In [23]:
# Trying to extract the range into a text file
select_range = list(range(775,831,1))

long_text = ''

for p in select_range:
    page = reader.pages[p]
    text = page.extract_text()
    long_text = long_text + text
    
print(len(long_text))




105090


In [28]:
# Write out to raw text
with open(source_text_output_path, 'w', encoding="utf-8") as f:
    f.write(long_text)

## Translation

In [49]:
import translators as ts


['DERTIENDE VERGADERIN']

In [64]:
import pandas as pd
trans_df = pd.DataFrame()

test_range = list(range(0,105_060, 30))


for i in test_range:
    ##parsed text
    parse_text = [long_text[i:i+31]]
    trans_df.loc[i,'input'] = parse_text



In [90]:
i=1
# print(''.join(trans_df.iloc[i]['input']))
sample_str = ''.join(trans_df.iloc[i]['input'])
print(sample_str)
print(type(sample_str))
# trans_df.iloc[i]['output']
output_str = ts.translate_text(sample_str,translator='google', from_language = 'nl', to_language='en')
# print(output_str)

ember 1874 .
BIBLIOTHEEK . HIST
<class 'str'>


In [92]:
print(output_str)

Ember 1874. 
 Library. Hist


In [72]:


for i in range(1,trans_df.shape[0]+1):
    trans_df.iloc[i]['output'] = ts.translate_text(trans_df.iloc[i]['input'],
    translator='google', 
    from_language = 'nl', 
    to_language='en')

TranslatorError: 